In [ ]:
## moving average
def movingAverage(data, windowSize):
    """ 
    where : 
    data : array of data values to be filtered
    windowSize : number of elements for moving average
    RETURSN : array of filtered values
    """
    moving = np.copy(data)
    
    avg = 0
    for i in range(0, windowSize):
        avg += moving[i]
    avg = avg / windowSize

    for i in range(0, len(moving)):
        if (i > windowSize):
            avg = avg + (data[i] - data[i-windowSize]) / windowSize
        moving[i] = avg
        
    return(moving)

    
## exponential filter (first order)
def firstOrder(data, tau, samplingRate = 1000, SP = 0):
    """ 
    where:
    data : array of flow values
    tau : filter time constant 
    samplingRate : data pts per sec
    RETURNS : array of filtered values
    """
    steptime = 1/samplingRate # sec
    filterfactor = 1 - np.exp(-steptime/tau)
    
    filtered = np.copy(data) # new array of filtered values to create waveform
    filtered[0] = SP
    filtered_last = filtered[0]
    for i in range(1, len(data)):
        filtered[i] = filtered_last * (1 - filterfactor) + filtered[i] * filterfactor
        filtered_last = filtered[i]
        
    return(filtered)

## linear interpolation - find x for given y between 2 pts
def linInterp(x0, y0, x1, y1, yStar):
    """
    where:
    [x0, y0] : first pt
    [x1, y1] : second pt
    yStar : value for interpolation
    RETURNS : x value for given yStar
    """
    m = (y1-y0) / (x1-x0)
    b = y0 - m * x0
    return(int(round((yStar-b)/m)))



In [ ]:
def pulseTransition(group, index, shift, cycleStart, MKStart, diff50, diffEnd):
    
    print('pulse module')
    searchTransition=0
    startTransition=0
    lastMidPtX=1
    lastMidPtY=1
    lastMidPtX2=1
    lastMidPtY2=1
    transition=0
    
    lastValue = 0
    for i in group.index[cycleStart:]:
        
        if index==4 or index==5 or index==6 or index==7 or index==8 or index==9 or index==12: # no pulsing, update code... TODO
            transition = cycleStart + 10000
            break
        
        if searchTransition == 0: # looking for transition to start
            if group['diff'][i] < (diffEnd+10):
                searchTransition = 1
        if searchTransition == 1: # start looking for up from pulse
            if group['diff'][i] > (diffEnd+50): # beginning of transition, possibly
                searchTransition = 2
                startTransition = i
                lastValue = group['diff'][i]
        if searchTransition == 2: # ensure that transition has occured (not just pulse below 1800)
            #if group['diff'][i] > (diff50 + 15):
            #    searchTransition = 0 # false alarm, start again
            if (group['diff'][i] < (diffEnd+20)): # or ((lastValue - group['diff'][i]) > 15):
                transition = startTransition
                transitionPulse = group['diff'][startTransition-shift:i-shift-50]
                half = len(transitionPulse) // 2
                transitionPulse = transitionPulse[half:] # look at last 2/3 of pulse
                transitionPulse.index = group[startTransition-shift+half:i-shift-50].index
                maxT = np.amax(transitionPulse[:])
                if (MKStart != 1500) and (group['MKS'][transitionPulse.index[0]]>1400):
                    #interpolate
                    print("Extrapolate")
                    transition = linInterp(lastMidPtX2, lastMidPtY2, lastMidPtX, lastMidPtY, diff50)
                    #transition_value = group['diff'][transition]
                    primaryDrying = group['datetime'][transition] - cycleData['Cycle Start Time'][index-1]
                    cycleData['pirani'][index-1] = primaryDrying
                    cycleData['Primary Duration'][index-1] = '%d hours %d minutes' % (primaryDrying.total_seconds()//3600, 
                                                                            (primaryDrying.total_seconds()%3600)//60)
                    cycleIndices['Pirani Transition'][index-1] = transition
                    cycleData['Pir dry end time'][index-1] = group['datetime'][transition]
                    searchTransition = 0
                    break
                elif (maxT > (diff50+15)) or (sum(transitionPulse<diff50)==0): # pulse above transition value, look to next pulse
                    searchTransition = 0
                    if len(transitionPulse) > 0:
                        lastMidPtX2 = lastMidPtX
                        lastMidPtY2 = lastMidPtY
                        lastMidPtX = transitionPulse.index[0]
                        lastMidPtY = transitionPulse[lastMidPtX]
                        
                elif maxT < diff50:
                    #interpolate
                    print("Interp")
                    midPtX = transitionPulse.index[0]
                    midPtY = transitionPulse[midPtX]
                    transition = linInterp(lastMidPtX, lastMidPtY, midPtX, midPtY, diff50)
                    #transition_value = group['diff'][transition]
                    primaryDrying = group['datetime'][transition] - cycleData['Cycle Start Time'][index-1]
                    cycleData['pirani'][index-1] = primaryDrying
                    cycleData['Primary Duration'][index-1] = '%d hours %d minutes' % (primaryDrying.total_seconds()//3600, 
                                                                            (primaryDrying.total_seconds()%3600)//60)
                    cycleIndices['Pirani Transition'][index-1] = transition
                    cycleData['Pir dry end time'][index-1] = group['datetime'][transition]
                    searchTransition = 0
                    break
                else: 
                    transition =np.argmax(transitionPulse<diff50)
                    print("transition")
                    transition_value = group['diff'][transition]
                    primaryDrying = group['datetime'][transition] - cycleData['Cycle Start Time'][index-1]
                    cycleData['pirani'][index-1] = primaryDrying
                    cycleData['Primary Duration'][index-1] = '%d hours %d minutes' % (primaryDrying.total_seconds()//3600, 
                                                                            (primaryDrying.total_seconds()%3600)//60)
                    cycleIndices['Pirani Transition'][index-1] = transition
                    cycleData['Pir dry end time'][index-1] = group['datetime'][transition]
                    searchTransition = 0
                    break
    return transition

def noPulseTransition(group, index, shift, cycleStart, MKStart, diff50, diffEnd):
    
    searchTransition=0
    startTransition=0
    lastMidPtX=0
    lastMidPtY=0
    lastMidPtX2=0
    transition=0
    
    lastValue = 0
    for i in group.index[cycleStart+3600:]: # start searching 5 hrs into cycle
        if group['diff'][i] <= diff50:
            print('no pulse: transition found')
            transition = i
            
            transition_value = group['diff'][transition]
            primaryDrying = group['datetime'][transition] - cycleData['Cycle Start Time'][index-1]
            cycleData['pirani'][index-1] = primaryDrying
            cycleData['Primary Duration'][index-1] = '%d hours %d minutes' % (primaryDrying.total_seconds()//3600, 
                                                                    (primaryDrying.total_seconds()%3600)//60)
            cycleIndices['Pirani Transition'][index-1] = transition
            cycleData['Pir dry end time'][index-1] = group['datetime'][transition]
            searchTransition = 0
            break
    return transition

In [ ]:
def determineTargetDiff(cycleStart):
    # initialize variables & dataframes
    pulseCount = 0
    pulseSearch = 0
    pulseStart = 0
    start = 0

    firstPulse = pd.DataFrame()
    secondPulse = pd.DataFrame()
    lastPulse = pd.DataFrame()
    secondLastPulse = pd.DataFrame()
    lastMKS = pd.DataFrame()
    secondLastMKS = pd.DataFrame()
    avgMKS = pd.DataFrame()
    avg1 = pd.DataFrame()
    avg2 = pd.DataFrame()
    avg1Diff = pd.DataFrame()
    avg2Diff = pd.DataFrame()
    lastDiff = pd.DataFrame()
    secondLastDiff = pd.DataFrame()
    lastMax = 0
    lastPulseIndex = 0
    dPirani = 0

    for i in group.index[cycleStart:]: # start seraching for first pulse at beginning of cycle

        if pulseSearch == 0: # not in pulse
            if group['Pirani'][i] < 450: # starting down pulse
                pulseSearch = 1
                if start == 0:
                    start = 1
                    next
                else:
                    if pulseCount == 1:
                        firstPulse = group['piraniFiltered'][pulseStart-shift:i-shift]
                        firstMKS = group['MKSFiltered'][pulseStart-shift:i-shift]
                        firstDiff = group['diffFiltered'][pulseStart-shift:i-shift]
                        max1 = np.amax(firstPulse[:]) 
                        maxM = np.amax(firstMKS[:])
                        maxDiff = np.amax(firstDiff[:])
                        firstPulse = firstPulse[firstPulse > (max1-10)]
                        firstMKS = firstMKS[firstMKS > (maxM-40)]
                        #firstDiff = firstDiff[firstDiff > (maxDiff-10)]
                        third = len(firstPulse)//3
                        avg1 = firstPulse[third:-20] # look at last 2/3 of hold, removing last 20 pts that start down pulse
                        avgMKS = firstMKS[third:-20]
                        avg1Diff = firstDiff[third:-20]
                    elif pulseCount == 2:
                        secondPulse = group['piraniFiltered'][pulseStart-shift:i-shift]
                        secondLastMKS = lastMKS
                        secondDiff = group['diffFiltered'][pulseStart-shift:i-shift]
                        lastMKS = group['MKS'][pulseStart-shift:i-shift]
                        max2 = np.amax(secondPulse[:])
                        maxM = np.amax(lastMKS[:])
                        max2Diff = np.amax(secondDiff[:])
                        secondPulse = secondPulse[secondPulse > (max2-10)]
                        third = len(secondPulse)//3
                        avg2 = secondPulse[third:-20]
                        avg2Diff = secondDiff[third:-20]
                        lastMax = max2
                        lastMaxDiff = max2Diff
                        lastPulseIndex = i
                        lastPulse = secondPulse
                        lastDiff = secondDiff
                        secondLastPulse = firstPulse
                        secondLastDiff = firstDiff
                    else: # look for final 2 pulses
                        check = group['piraniFiltered'][pulseStart-shift:i-shift]
                        checkM = group['MKS'][pulseStart-shift:i-shift]
                        checkDiff = group['diffFiltered'][pulseStart-shift:i-shift]
                        maxi = np.amax(check[:])
                        maxM = np.amax(checkM[:])
                        maxiDiff = np.amax(checkDiff[:])
                        if len(check) < 200:
                            break
                        else:
                            third = len(check)//3
                            check = check[third:-20]
                            checkDiff = checkDiff[third:-20]
                            secondLastPulse = lastPulse
                            secondLastMKS = lastMKS
                            secondLastDiff = lastDiff
                            lastPulse = check
                            lastMKS = checkM
                            lastDiff = checkDiff
                            lastMax = maxi
                            lastMaxDiff = maxiDiff
                            lastPulseIndex = i - shift

        if pulseSearch == 1:
            if group['Pirani'][i] > 500:
                pulseSearch = 0
                pulseStart = i
                pulseCount += 1

    pulseSearch = 0
    startDrop = 0
    drop = pd.DataFrame()
    dropPirani = pd.DataFrame()
    for i in group.index[lastPulseIndex:]: # search for point where MKS is 1000 mTorr
        if pulseSearch == 0:
            if group['MKS'][i] < 1000:
                pulseSearch = 1
                startDrop = i
        if pulseSearch == 1:
            if group['MKS'][i] < 850: # pulse, not drop to 
                pulseSearch = 2
            elif group['MKS'][i] > 1050:
                drop = group['MKS'][startDrop-shift:i-1-shift]
                dropPirani = group['piraniFiltered'][startDrop-shift:i-1-shift]
                dPirani = group['piraniFiltered'][startDrop]
                break
        if pulseSearch == 2:
            if group['MKS'][i] > 1050: # restart drop search
                pulseSearch = 0
                
    MKStart = np.average(avgMKS)
    MKStart = round(MKStart/100) * 100
    #print(MKStart)
    pulseSearch = 0
    for i in group.index[startDrop::-1]: # search backwards from where MKS is 1000 mTorr to where it equals start MKS
        if pulseSearch == 0:
            if group['MKS'][i] < 400:
                pulseSearch = 1
        if pulseSearch == 1:
            if group['MKS'][i] > MKStart:
                dPirani = group['diff'][i+1]
                break



    PiraniStartPV = (np.average(avg1) + np.average(avg2)) / 2
    PiraniEndPV = np.average(lastMKS)
    diff50 = (PiraniStartPV - PiraniEndPV) / 2 // .1 /10
    piraniData['Pirani Start'][index-1] = PiraniStartPV // .1 /10 # round to 1 decimal pt
    piraniData['Pirani End'][index-1] = PiraniEndPV // .1 /10
    piraniData['diffStart'][index-1] = ((np.average(avg1Diff)+np.average(avg2Diff))/2) // 0.1 / 10
    piraniData['diffEnd'][index-1] = ((np.average(lastDiff)+np.average(secondLastDiff))/2) // 0.1 / 10
    piraniData['diff50'][index-1] = diff50 
    piraniData['MKS_Start'][index-1] = MKStart

    #offset = (np.average(dropPirani) - np.average(drop)) // .1 /10
    offset = dPirani
    piraniData['Offset'][index-1] = offset
    #piraniData['diffEnd'][index-1] = offset
    target = diff50 + offset
    piraniData['Target'][index-1] = target
    piraniData['Target2'][index-1] = (piraniData['diffStart'][index-1]+piraniData['Offset'][index-1])/2
    
    return piraniData['Target2'][index-1]

In [ ]:
def determineTargetDiffNoPulse(cycleStart):
    # initialize variables & dataframes
    print('no pulse')
    
    avgMKS = group['MKS'][cycleStart+2000:cycleStart+5000]
    avgPirani = group['Pirani'][cycleStart+2000:cycleStart+5000]
    avgDiff = group['diff'][cycleStart+2000:cycleStart+5000]
                
    MKStart = np.average(avgMKS)
    MKStart = round(MKStart/50) * 50

    for i in group.index[-2::-1]: # search backwards from end to value of start MKS
        if group['MKS'][i] < MKStart:
            dPirani = group['diff'][i]
            break
        
    PiraniStartPV = np.average(avgPirani)
    #PiraniEndPV = np.average(lastMKS)
    #diff50 = (PiraniStartPV - PiraniEndPV) / 2 // .1 /10
    piraniData['Pirani Start'][index-1] = PiraniStartPV // .1 /10 # round to 1 decimal pt
    piraniData['Pirani End'][index-1] = -100
    piraniData['diffStart'][index-1] = np.average(avgDiff)
    piraniData['diffEnd'][index-1] = dPirani
    piraniData['diff50'][index-1] = (piraniData['diffStart'][index-1] + piraniData['diffEnd'][index-1])/2 
    piraniData['MKS_Start'][index-1] = MKStart

    #offset = (np.average(dropPirani) - np.average(drop)) // .1 /10
    offset = dPirani
    piraniData['Offset'][index-1] = offset
    #piraniData['diffEnd'][index-1] = offset
    target = piraniData['diff50'][index-1] + offset
    piraniData['Target'][index-1] = -100
    piraniData['Target2'][index-1] = (piraniData['diffStart'][index-1]+piraniData['Offset'][index-1])/2
    
    return piraniData['Target2'][index-1]

In [ ]:
def determineTargetDiffOld():
    # initialize variables & dataframes
    pulseCount = 0
    pulseSearch = 0
    pulseStart = 0
    start = 0

    firstPulse = pd.DataFrame()
    secondPulse = pd.DataFrame()
    lastPulse = pd.DataFrame()
    secondLastPulse = pd.DataFrame()
    lastMKS = pd.DataFrame()
    secondLastMKS = pd.DataFrame()
    avgMKS = pd.DataFrame()
    avg1 = pd.DataFrame()
    avg2 = pd.DataFrame()
    avg1Diff = pd.DataFrame()
    avg2Diff = pd.DataFrame()
    lastDiff = pd.DataFrame()
    secondLastDiff = pd.DataFrame()
    lastMax = 0
    lastPulseIndex = 0
    dPirani = 0

    for i in group.index[cycleStart:]: # start seraching for first pulse at beginning of cycle

        if pulseSearch == 0: # not in pulse
            if group['Pirani'][i] < 450: # starting down pulse
                pulseSearch = 1
                if start == 0:
                    start = 1
                    next
                else:
                    pulseCount += 1
                    if pulseCount == 1:
                        firstPulse = group['piraniFiltered'][pulseStart-shift:i-shift]
                        firstMKS = group['MKSFiltered'][pulseStart-shift:i-shift]
                        firstDiff = group['diffFiltered'][pulseStart-shift:i-shift]
                        max1 = np.amax(firstPulse[:]) 
                        maxM = np.amax(firstMKS[:])
                        maxDiff = np.amax(firstDiff[:])
                        firstPulse = firstPulse[firstPulse > (max1-10)]
                        firstMKS = firstMKS[firstMKS > (maxM-40)]
                        #firstDiff = firstDiff[firstDiff > (maxDiff-10)]
                        third = len(firstPulse)//3
                        avg1 = firstPulse[third:-20] # look at last 2/3 of hold, removing last 20 pts that start down pulse
                        avgMKS = firstMKS[third:-20]
                        avg1Diff = firstDiff[third:-20]
                    elif pulseCount == 2:
                        secondPulse = group['piraniFiltered'][pulseStart-shift:i-shift]
                        secondLastMKS = lastMKS
                        secondDiff = group['diffFiltered'][pulseStart-shift:i-shift]
                        lastMKS = group['MKS'][pulseStart-shift:i-shift]
                        max2 = np.amax(secondPulse[:])
                        maxM = np.amax(lastMKS[:])
                        max2Diff = np.amax(secondDiff[:])
                        secondPulse = secondPulse[secondPulse > (max2-10)]
                        third = len(secondPulse)//3
                        avg2 = secondPulse[third:-20]
                        avg2Diff = secondDiff[third:-20]
                        lastMax = max2
                        lastMaxDiff = max2Diff
                        lastPulseIndex = i
                        lastPulse = secondPulse
                        lastDiff = secondDiff
                        secondLastPulse = firstPulse
                        secondLastDiff = firstDiff
                    else: # look for final 2 pulses
                        check = group['piraniFiltered'][pulseStart-shift:i-shift]
                        checkM = group['MKS'][pulseStart-shift:i-shift]
                        checkDiff = group['diffFiltered'][pulseStart-shift:i-shift]
                        maxi = np.amax(check[:])
                        maxM = np.amax(checkM[:])
                        maxiDiff = np.amax(checkDiff[:])
                        if len(check) < 200:
                            break
                        else:
                            third = len(check)//3
                            check = check[third:-20]
                            checkDiff = checkDiff[third:-20]
                            secondLastPulse = lastPulse
                            secondLastMKS = lastMKS
                            secondLastDiff = lastDiff
                            lastPulse = check
                            lastMKS = checkM
                            lastDiff = checkDiff
                            lastMax = maxi
                            lastMaxDiff = maxiDiff
                            lastPulseIndex = i - shift

        if pulseSearch == 1:
            if group['Pirani'][i] > 500:
                pulseSearch = 0
                pulseStart = i

    pulseSearch = 0
    startDrop = 0
    drop = pd.DataFrame()
    dropPirani = pd.DataFrame()
    for i in group.index[lastPulseIndex:]: # search for point where MKS is 1000 mTorr
        if pulseSearch == 0:
            if group['MKS'][i] < 1000:
                pulseSearch = 1
                startDrop = i
        if pulseSearch == 1:
            if group['MKS'][i] < 850: # pulse, not drop to 
                pulseSearch = 2
            elif group['MKS'][i] > 1050:
                drop = group['MKS'][startDrop-shift:i-1-shift]
                dropPirani = group['piraniFiltered'][startDrop-shift:i-1-shift]
                dPirani = group['piraniFiltered'][startDrop]
                break
        if pulseSearch == 2:
            if group['MKS'][i] > 1050: # restart drop search
                pulseSearch = 0
                
    MKStart = np.average(avgMKS)
    MKStart = round(MKStart/100) * 100
    #print(MKStart)
    pulseSearch = 0
    for i in group.index[startDrop::-1]: # search backwards from where MKS is 1000 mTorr to where it equals start MKS
        if pulseSearch == 0:
            if group['MKS'][i] < 400:
                pulseSearch = 1
        if pulseSearch == 1:
            if group['MKS'][i] > MKStart:
                dPirani = group['diff'][i+1]
                break



    PiraniStartPV = (np.average(avg1) + np.average(avg2)) / 2
    PiraniEndPV = np.average(lastMKS)
    diff50 = (PiraniStartPV - PiraniEndPV) / 2 // .1 /10
    piraniData['Pirani Start'][index-1] = PiraniStartPV // .1 /10 # round to 1 decimal pt
    piraniData['Pirani End'][index-1] = PiraniEndPV // .1 /10
    piraniData['diffStart'][index-1] = ((np.average(avg1Diff)+np.average(avg2Diff))/2) // 0.1 / 10
    piraniData['diffEnd'][index-1] = ((np.average(lastDiff)+np.average(secondLastDiff))/2) // 0.1 / 10
    piraniData['diff50'][index-1] = diff50 
    piraniData['MKS_Start'][index-1] = MKStart

    #offset = (np.average(dropPirani) - np.average(drop)) // .1 /10
    offset = dPirani
    piraniData['Offset'][index-1] = offset
    #piraniData['diffEnd'][index-1] = offset
    target = diff50 + offset
    piraniData['Target'][index-1] = target
    piraniData['Target2'][index-1] = (piraniData['diffStart'][index-1]+piraniData['Offset'][index-1])/2
    


In [ ]:
print('pulse functions loaded')